In [1]:
import random
def generate_theme_title_and_story(campaign_title: str, insight: str) -> tuple[str, str]:
    title_templates = [
        "The Power of {keyword}",
        "Mastering {keyword} for Growth",
        "Why {keyword} Matters Now",
        "{keyword}: A New Approach",
        "Unlocking {keyword}"
    ]

    insight_keywords = insight.split()
    keyword = random.choice(insight_keywords if insight_keywords else ["change"])
    title = random.choice(title_templates).format(keyword=keyword.capitalize())

    story = f"This theme explores how '{keyword}' relates to {campaign_title.lower()}. We’ll break down why this is crucial and how to apply it practically."
    return title, story
generate_theme_title_and_story("123", "123")

('123: A New Approach',
 "This theme explores how '123' relates to 123. We’ll break down why this is crucial and how to apply it practically.")

In [ ]:
import asyncio


async def generate_topic():
    response = await openai.ChatCompletion.acreate(
        model="deepseek-chat",
        messages=[
            {"role": "system", "content": "You are a creative assistant."},
            {"role": "user", "content": "Generate a unique topic for a technology blog."}
        ]
    )
    topic = response.choices[0].message['content']
    print(f"Generated Topic: {topic}")

# Run the asynchronous function
asyncio.run(generate_topic())

In [ ]:
from openai import OpenAI
import asyncio
from dotenv import load_dotenv
import nest_asyncio
import pandas as pd
import time
import os

# Load environment variables and apply nest_asyncio
load_dotenv()
nest_asyncio.apply()

# Initialize OpenAI client with DeepSeek endpoint
client = OpenAI(api_key=os.getenv("DEEPSEEK_API_KEY"), base_url="https://api.deepseek.com")

# Define the topics for the posts
topics = [
    "Benefits of meditation",
    "How to start programming",
    "Climate change solutions",
    "Healthy meal prep ideas",
    "Financial planning tips",
    "Productivity hacks",
    "Travel on a budget",
    "Home workout routines",
    "Book recommendations for beginners",
    "Tech trends 2025"
]

async def generate_post(topic):
    """Generate a post about a specific topic using DeepSeek."""
    try:
        response = client.chat.completions.create(
            model="deepseek-chat",
            messages=[
                {"role": "system", "content": "You are a helpful assistant that writes blog posts"},
                {"role": "user", "content": f"Write a short blog post about {topic}. Keep it under 300 words."},
            ],
            stream=False
        )
        content = response.choices[0].message.content
        
        print(f"Generated post for: {topic}")
        
        return {
            "topic": topic,
            "content": content,
            "timestamp": time.strftime("%Y-%m-%d %H:%M:%S")
        }
    except Exception as e:
        print(f"Error generating post for {topic}: {e}")
        return {
            "topic": topic,
            "content": f"Error: {str(e)}",
            "timestamp": time.strftime("%Y-%m-%d %H:%M:%S")
        }

async def process_topics():
    """Process all topics concurrently."""
    tasks = []
    for topic in topics:
        task = asyncio.create_task(generate_post(topic))
        tasks.append(task)
    
    results = await asyncio.gather(*tasks)
    return results

async def main():
    print("Generating 10 posts asynchronously with DeepSeek...")
    start_time = time.time()
    
    results = await process_topics()
    
    # Create DataFrame from results
    df = pd.DataFrame(results)
    
    # Display summary of the DataFrame
    print("\nGenerated posts summary:")
    for index, row in df.iterrows():
        print(f"{index+1}. {row['topic']} - {len(row['content'])} chars")
    
    # Export to Excel
    excel_filename = "deepseek_generated_posts.xlsx"
    df.to_excel(excel_filename, index=False)
    
    end_time = time.time()
    print(f"\nExported {len(results)} posts to {excel_filename}")
    print(f"Total time: {end_time - start_time:.2f} seconds")

if __name__ == "__main__":
    asyncio.run(main())

Generating 10 posts asynchronously with DeepSeek...
Generated post for: Benefits of meditation
Generated post for: How to start programming
Generated post for: Climate change solutions
Generated post for: Healthy meal prep ideas
Generated post for: Financial planning tips
Generated post for: Productivity hacks
Generated post for: Travel on a budget
Generated post for: Home workout routines
Generated post for: Book recommendations for beginners
Generated post for: Tech trends 2025

Generated posts summary:
1. Benefits of meditation - 1359 chars
2. How to start programming - 1472 chars
3. Climate change solutions - 1494 chars
4. Healthy meal prep ideas - 1511 chars
5. Financial planning tips - 1394 chars
6. Productivity hacks - 1185 chars
7. Travel on a budget - 1454 chars
8. Home workout routines - 1395 chars
9. Book recommendations for beginners - 1418 chars
10. Tech trends 2025 - 1705 chars


ModuleNotFoundError: No module named 'openpyxl'

In [ ]:
import os
import json
import asyncio
import time
import pandas as pd
from dotenv import load_dotenv
import nest_asyncio
from google import genai
from google.genai import types
from typing import List, Dict, Any
from pydantic import BaseModel, ValidationError

# Load environment variables and apply nest_asyncio
load_dotenv()
nest_asyncio.apply()

# Define topics for generation
topics = [
    "Benefits of meditation",
    "How to start programming",
    "Climate change solutions",
    "Healthy meal prep ideas",
    "Financial planning tips",
    "Productivity hacks",
    "Travel on a budget",
    "Home workout routines",
    "Book recommendations for beginners",
    "Tech trends 2025"
]


# System prompt - adapt this to your needs
system_prompt = """
You are an assistant that writes engaging blog posts. 
Create content that is informative, well-structured, and under 300 words.
Format your response as a JSON object with 'title' and 'content' fields.
"""

class BlogPost(BaseModel):
    title: str
    content: str

async def generate_post_with_gemini(topic: str) -> Dict[str, Any]:
    """Generate a post about a specific topic using Google Gemini."""
    try:
        # Initialize the client similar to your implementation
        client = genai.Client(api_key=os.getenv("GEMINI_API_KEY") )
        
        # Create the prompt
        prompt = f"Write a short blog post about {topic}. Keep it under 300 words."
        
        # Generate response using Gemini API - following your implementation pattern
        response = client.models.generate_content(
            model='gemini-2.0-flash',
            contents=prompt,
            config={
                'response_mime_type': 'application/json',
                'response_schema': BlogPost,
                'system_instruction': types.Part.from_text(text=system_prompt),
            },
        )
        
        # Extract the response - similar to your approach
        print(f"Generated post for: {topic}")
        content = json.loads(response.text)
        blog_post = BlogPost(**content)
        
        return {
            "topic": topic,
            "title": blog_post.title,
            "content": blog_post.content,
            "timestamp": time.strftime("%Y-%m-%d %H:%M:%S")
        }
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Gemini API Error: {str(e)}")


# This function handles creating the semaphore and processing tasks
async def process_with_semaphore(topics: List[str], concurrency: int = 10):
    """Process topics with a semaphore to limit concurrent API calls."""
    # Use a semaphore to control concurrency
    semaphore = asyncio.Semaphore(concurrency)
    
    async def bounded_generate(topic):
        async with semaphore:
            return await generate_post_with_gemini(topic)
    
    tasks = [bounded_generate(topic) for topic in topics]
    return await asyncio.gather(*tasks)

async def main():
    print("Generating posts asynchronously with Google Gemini...")
    start_time = time.time()
    
    # Process all topics with controlled concurrency
    results = await process_with_semaphore(topics)
    
    # Create a DataFrame from the results
    df = pd.DataFrame(results)
    
    # Display summary of the DataFrame
    print("\nGenerated posts summary:")
    for index, row in df.iterrows():
        print(f"{index+1}. {row['title']} - {len(row['content'])} chars")
    
    # Export to Excel
    excel_filename = "gemini_generated_posts.xlsx"
    df.to_excel(excel_filename, index=False)
    
    end_time = time.time()
    print(f"\nExported {len(results)} posts to {excel_filename}")
    print(f"Total time: {end_time - start_time:.2f} seconds")

if __name__ == "__main__":
    asyncio.run(main())

Generating posts asynchronously with Google Gemini...
Generated post for: Benefits of meditation
Generated post for: How to start programming
Generated post for: Climate change solutions
Generated post for: Healthy meal prep ideas
Generated post for: Financial planning tips
Generated post for: Productivity hacks
Generated post for: Travel on a budget
Generated post for: Home workout routines
Generated post for: Book recommendations for beginners
Generated post for: Tech trends 2025

Generated posts summary:
1. Unlock Inner Peace: The Benefits of Meditation - 1348 chars
2. Embark on Your Coding Journey: A Beginner's Guide - 1076 chars
3. Fighting Back: Practical Climate Change Solutions - 1221 chars
4. Fuel Your Week: Easy & Healthy Meal Prep Ideas - 1209 chars
5. Smart Financial Planning Tips for a Secure Future - 791 chars
6. Boost Your Day: Simple Productivity Hacks - 900 chars
7. Travel on a Budget: See the World Without Breaking the Bank - 1239 chars
8. Effective Home Workout Routi

In [15]:
import os
import json
import time
from typing import Dict, Any
from pydantic import BaseModel, ValidationError
from google import genai
from google.genai import types

# Load environment variables
from dotenv import load_dotenv
load_dotenv()

# System prompt - adapt this to your needs
system_prompt = """
You are an assistant that writes engaging blog posts. 
Create content that is informative, well-structured, and under 300 words.
Format your response as a JSON object with 'title' and 'content' fields.
"""

class BlogPost(BaseModel):
    title: str
    content: str

async def generate_post_with_gemini(user_prompt: str) -> Dict[str, Any]:
    """Generate a post based on a user-provided prompt using Google Gemini."""
    try:
        # Initialize the client
        client = genai.Client(api_key=os.getenv("GEMINI_API_KEY"))
        
        # Generate response using Gemini API
        response = await client.aio.models.generate_content(
            model='gemini-2.0-flash',
            contents=user_prompt,
            config=types.GenerateContentConfig(
                response_mime_type='application/json',
                response_schema=BlogPost,
                system_instruction=types.Part.from_text(text=system_prompt),
            ),
        )
        
        # Extract the response
        print("Generated post based on user prompt.")
        content = json.loads(response.text)
        
        # Validate and parse the response using Pydantic
        blog_post = BlogPost(**content)
        
        return {
            "title": blog_post.title,
            "content": blog_post.content,
            "timestamp": time.strftime("%Y-%m-%d %H:%M:%S")
        }
    except ValidationError as ve:
        print(f"Validation error: {ve}")
        return {
            "title": "Validation Error",
            "content": f"Validation Error: {str(ve)}",
            "timestamp": time.strftime("%Y-%m-%d %H:%M:%S")
        }
    except Exception as e:
        print(f"Error generating post: {e}")
        return {
            "title": "Error",
            "content": f"Error: {str(e)}",
            "timestamp": time.strftime("%Y-%m-%d %H:%M:%S")
        }
user_prompt = "Write a short blog post about the benefits of meditation. Keep it under 300 words."

# Run the asynchronous function
result = asyncio.run(generate_post_with_gemini(user_prompt))

# Display the result
print("Title:", result["title"])
print("Content:", result["content"])
print("Timestamp:", result["timestamp"])

Generated post based on user prompt.
Title: Unlocking Inner Peace: The Benefits of Meditation
Content: In today's fast-paced world, finding moments of calm can feel like a luxury. However, meditation offers a simple yet powerful way to cultivate inner peace and improve overall well-being. Regular meditation practice provides numerous benefits for both your mind and body.

One of the most significant advantages is stress reduction. Meditation helps calm the nervous system, lowering cortisol levels and promoting relaxation. This can lead to reduced anxiety and a greater sense of emotional balance. Beyond stress relief, meditation enhances focus and concentration. By training your mind to stay present, you can improve your ability to concentrate on tasks and make better decisions.

Furthermore, meditation can positively impact your physical health. Studies have shown that it can lower blood pressure, improve sleep quality, and even boost your immune system. It also promotes self-awareness